In [ ]:
import os
import requests
import pandas as pd
import urllib3
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from PIL import Image, UnidentifiedImageError
from io import BytesIO
import warnings

In [ ]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
warnings.simplefilter("ignore", UserWarning)

dataset_path = './dataset/'

if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)
else:
    print(f"The folder '{dataset_path}' already exists.")

with open("logs.txt", "a") as logs_file:
    pass

In [ ]:
def save_image(url, file_name, dataset_path):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.49 Safari/537.36 Edge/112.0.1722.64',
            'Accept-Language': 'en-US,en;q=0.5',
        }

        try:
            response = requests.get(url, headers=headers, timeout=10, verify=False)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            if 'favicon.ico' not in url:
                with open("logs.txt", "a") as logs_file:
                    logs_file.write(f"Error accessing {url}: {e}\n")
            return None

        content_type = response.headers.get('Content-Type', '')
        valid_types = ['image/png', 'image/jpeg', 'image/jpg', 'image/webp', 'image/gif', 'image/x-icon', 'image/vnd.microsoft.icon']
        if content_type not in valid_types:
            return None

        image_data = BytesIO(response.content)
        try:
            image = Image.open(image_data)
            image.verify()
            image = Image.open(image_data)
        except UnidentifiedImageError:
            return None

        width, height = image.size
        if width > 512 or height > 512:
            return None

        image = image.convert("RGBA")
        file_path = os.path.join(dataset_path, file_name + ".png")
        image.save(file_path, format="PNG")

    except Exception as e:
        with open("logs.txt", "a") as logs_file:
            logs_file.write(f"Error downloading the image from {url}: {e}\n")

In [ ]:
def get_logo_from_html(domain):
    response = None
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.49 Safari/537.36 Edge/112.0.1722.64',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept': 'image/x-icon, image/*;q=0.8, */*;q=0.5'
        }

        urls = [
            f'https://www.{domain}',
            f'http://www.{domain}',
            f'https://{domain}',
            f'http://{domain}'
        ]

        correctUrl = ''
        for url in urls:
            headers['Referer'] = url
            response = requests.get(url, verify=False, headers=headers, timeout=10)
                    
            if response.status_code == 403:
                correctUrl = url
                break

            response.raise_for_status()
            correctUrl = url
            break

        if response is None:
            with open("logs.txt", "a") as logs_file:
                logs_file.write(f"All requests failed for {domain}\n")
            return None, []

        try:
            soup = BeautifulSoup(response.text, 'html.parser')

            found_images = []
            images = soup.find_all('img')
            for image in images:
                image_url = image.get('src')
                alt_text = image.get('alt', '').lower()
                file_name = os.path.basename(image_url) if image_url else ''

                if image.find_parent('header') and (
                    'logo' in alt_text or 
                    'logo' in file_name.lower() or 
                    'favicon' in file_name.lower()
                ):
                    found_images.append(image_url)

            link_tags = soup.find_all('link', rel='icon') + soup.find_all('link', rel='shortcut icon')
            for link in link_tags:
                found_images.append(link.get('href'))
            
            found_images.append('/favicon.ico')
            
            found_images = list(set(filter(None, found_images)))
            return correctUrl, found_images
        
        except Exception as e:
            with open("logs.txt", "a") as logs_file:
                logs_file.write(f"Parsing error for {domain}: {e}\n")
            return None, []
    except Exception as e:
        with open("logs.txt", "a") as logs_file:
            logs_file.write(f"Error processing domain {domain}: {e}\n")
        return None, []

In [ ]:
def show_progress(current, total, last_displayed_progress):
    progress = (current + 1) / total * 100
    progress = (progress // 10) * 10

    if progress >= last_displayed_progress + 10:
        print(f"Progress: {progress}% ({current + 1}/{total})")
        return progress
    
    return last_displayed_progress

def scrape_from_domains(domains):
    size = len(domains)
    last_progress = -1

    for i, domain in enumerate(domains):
        file_name = domain + '_' + str(i)

        last_progress = show_progress(i, size, last_progress)

        try:
            url, logo_urls = get_logo_from_html(domain)
            
            for j, logo_url in enumerate(logo_urls):
                logo_url = urljoin(url, logo_url)
                save_image(logo_url, f"{file_name}_{j}", dataset_path)
        except Exception as e:
            with open("logs.txt", "a") as logs_file:
                logs_file.write(f"Error processing domain {domain}: {e}\n")

In [ ]:
df = pd.read_parquet("logos.snappy.parquet", engine="pyarrow")

domains = df['domain']

scrape_from_domains(domains)